<a href="https://colab.research.google.com/github/taras-musakovskyi/colab-jupyter-fish-models/blob/main/YOLO_fish5_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Google Colab YOLOv8 Fish Detection Training


# ============================================================================
# CELL 1: Setup and Installation
# ============================================================================
!pip install ultralytics roboflow

import os
from google.colab import drive, files
from ultralytics import YOLO
import torch
import yaml


# Check GPU availability
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.7/88.7 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 103.3 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings wit

In [ ]:
# ============================================================================
# CELL 2: Dataset Upload and Preparation
# ============================================================================

uploaded = files.upload()

# Extract the dataset
import zipfile
zip_filename = list(uploaded.keys())[0]  # Get the uploaded zip filename

# Extract to a known location
dataset_path = "/content/fish_dataset"
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(dataset_path)

print(f"Dataset extracted to: {dataset_path}")

# List contents to verify structure
import os
print("\nDataset structure:")
for root, dirs, files in os.walk(dataset_path):
    level = root.replace(dataset_path, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    sub_indent = ' ' * 2 * (level + 1)
    for file in files[:3]:  # Show first 3 files only
        print(f"{sub_indent}{file}")
    if len(files) > 3:
        print(f"{sub_indent}... and {len(files)-3} more files")

Saving FishAnnotations.v2i.yolov8.with.group.weights.zip to FishAnnotations.v2i.yolov8.with.group.weights.zip
Dataset extracted to: /content/fish_dataset

Dataset structure:
fish_dataset/
  Fish Annotation.v2i.yolov8/
    README.roboflow.txt
    README.dataset.txt
    data.yaml
    test/
      images/
        frame_0027_jpg.rf.cd8f15765b1a82bc826f89cdf751cf26.jpg
        frame_0069_jpg.rf.428652d0c3fa20673878124e96ef4b07.jpg
        frame_0250_jpg.rf.c15df06c22d86565fa68162cd19a5e3c.jpg
        ... and 60 more files
      labels/
        2-frame_00037_t0003_000_q0_833_jpg.rf.11a3ee1f922c3db4b966104af44b74cb.txt
        frame_0069_jpg.rf.9963214bedc472a3b9e38f45430dc432.txt
        frame_0250_jpg.rf.c15df06c22d86565fa68162cd19a5e3c.txt
        ... and 60 more files
    valid/
      images/
        frame_0056_jpg.rf.9cfe43f7b31a98ab0c309e03999bc83f.jpg
        other1_frame_00340_t0028_666_q0_833_jpg.rf.0581b80e8290c81e6677346f8d47d5d5.jpg
        frame_02670_t0231_150_q0_833_jpg.rf.a0b99

In [ ]:
# ============================================================================
# CELL 3: Move Dataset Contents Up and Create Configuration
# ============================================================================

import os
import shutil

# Define paths
dataset_path = "fish_dataset"  # Adjust if different
inner_dataset_path = f"{dataset_path}/Fish Annotation.v2i.yolov8"

# Move contents from inner folder to main dataset folder
if os.path.exists(inner_dataset_path):
    print("Moving dataset contents up...")

    # Move all contents from inner folder to main folder
    for item in os.listdir(inner_dataset_path):
        src = os.path.join(inner_dataset_path, item)
        dst = os.path.join(dataset_path, item)

        # Remove destination if it exists (handles duplicate data.yaml)
        if os.path.exists(dst):
            if os.path.isdir(dst):
                shutil.rmtree(dst)
            else:
                os.remove(dst)

        # Move item
        shutil.move(src, dst)

    # Remove empty inner folder
    os.rmdir(inner_dataset_path)
    print("✓ Dataset structure flattened")

# Create data.yaml file with class weights
data_yaml_fixed = f"""
path: {dataset_path}  # dataset root dir
train: train/images  # train images (relative to 'path')
val: valid/images    # val images (relative to 'path')
test: test/images    # test images (relative to 'path')

nc: 8
names: ['ancistrus', 'black molly', 'dalmatian molly', 'gold fish', 'gold molly', 'guppy female', 'guppy male', 'helena']

class_weights: [3.0, 2.0, 8.0, 2.5, 1.0, 1.8, 8.5, 5.5]
"""

# Save fixed data.yaml
with open(f'{dataset_path}/data.yaml', 'w') as f:
    f.write(data_yaml_fixed)

print("✓ data.yaml updated with correct paths and class weights:")
print(data_yaml_fixed)

# Verify final structure
print("\n✓ Final dataset structure:")
for root, dirs, files in os.walk(dataset_path):
    level = root.replace(dataset_path, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f'{indent}{os.path.basename(root)}/')
    subindent = ' ' * 2 * (level + 1)
    for file in files[:5]:  # Show first 5 files only
        print(f'{subindent}{file}')
    if len(files) > 5:
        print(f'{subindent}... and {len(files)-5} more files')

Moving dataset contents up...
✓ Dataset structure flattened
✓ data.yaml updated with correct paths and class weights:

path: fish_dataset  # dataset root dir
train: train/images  # train images (relative to 'path')
val: valid/images    # val images (relative to 'path')
test: test/images    # test images (relative to 'path')

nc: 8
names: ['ancistrus', 'black molly', 'dalmatian molly', 'gold fish', 'gold molly', 'guppy female', 'guppy male', 'helena']

class_weights: [3.0, 2.0, 8.0, 2.5, 1.0, 1.8, 8.5, 5.5]


✓ Final dataset structure:
fish_dataset/
  README.roboflow.txt
  README.dataset.txt
  data.yaml
  test/
    images/
      frame_0027_jpg.rf.cd8f15765b1a82bc826f89cdf751cf26.jpg
      frame_0069_jpg.rf.428652d0c3fa20673878124e96ef4b07.jpg
      frame_0250_jpg.rf.c15df06c22d86565fa68162cd19a5e3c.jpg
      frame_0143_jpg.rf.2e4b31131a6007865a8cc0f32f025dba.jpg
      frame_0005_jpg.rf.d952512466546d2600bb700319de7069.jpg
      ... and 58 more files
    labels/
      2-frame_00037_t0003

In [ ]:
#check
# Check ALL label files to find all classes
import glob
print(dataset_path)
label_files = glob.glob(f'{dataset_path}/train/labels/*.txt')
print(f"Checking all {len(label_files)} label files...")

class_ids = set()
class_counts = {}

for label_file in label_files:
    with open(label_file, 'r') as f:
        for line in f:
            if line.strip():
                class_id = int(line.strip().split()[0])
                class_ids.add(class_id)
                class_counts[class_id] = class_counts.get(class_id, 0) + 1

print(f"\nAll class IDs found: {sorted(class_ids)}")
print(f"Total classes: {len(class_ids)}")

print("\nClass distribution:")
for class_id in sorted(class_counts.keys()):
    species_name = ['ancistrus', 'black molly', 'dalmatian molly', 'gold fish', 'gold molly', 'guppy female', 'guppy male', 'helena'][class_id]
    print(f"Class {class_id} ({species_name}): {class_counts[class_id]} instances")

fish_dataset
Checking all 1371 label files...

All class IDs found: [0, 1, 2, 3, 4, 5, 6, 7]
Total classes: 8

Class distribution:
Class 0 (ancistrus): 555 instances
Class 1 (black molly): 1059 instances
Class 2 (dalmatian molly): 226 instances
Class 3 (gold fish): 791 instances
Class 4 (gold molly): 2379 instances
Class 5 (guppy female): 1149 instances
Class 6 (guppy male): 210 instances
Class 7 (helena): 375 instances


In [ ]:
# ============================================================================
# CELL 5: Train XLarge YOLO Model
# ============================================================================

# Available YOLO model sizes
model_variants = {
    'nano': 'yolov8n.pt'#,     # Fastest, smallest
    #'small': 'yolov8s.pt',    # Good balance
    #'medium': 'yolov8m.pt',   # Better accuracy
    #'large': 'yolov8l.pt',    # High accuracy
    #'xlarge': 'yolov8x.pt'    # Best accuracy, slowest
}

# Choose which models to train (comment out ones you don't want)
models_to_train = [
    'nano'#,    # ~6 minutes training
#    'small',   # ~12 minutes training
#    'medium',  # ~25 minutes training
#    'large',   # ~45 minutes training (might hit Colab timeout)
#     'xlarge', # ~60+ minutes (likely to timeout in free Colab)
]

trained_results = {}

for model_name in models_to_train:
    print(f"\n{'='*60}")
    print(f"🚀 TRAINING YOLO{model_name.upper()} MODEL")
    print('='*60)

    # Load the model
    model = YOLO(model_variants[model_name])

    # Adjust batch size based on model size
    batch_sizes = {
        'nano': 16,
        'small': 12,
        'medium': 8,
        'large': 4,
        'xlarge': 2
    }

    batch_size = batch_sizes[model_name]
    print(f"Using batch size: {batch_size}")

    # Train the model
    results = model.train(
        data=f'{dataset_path}/data.yaml',
        epochs=50,           # Reduced epochs for multiple models
        imgsz=640,          # Image size
        batch=batch_size,   # Adjusted batch size
    device=0,           # GPU device (0 for first GPU)
    project='fish_detection',  # Project name
    name='yolov8n_fish_v1',    # Run name

    # Training hyperparameters
    lr0=0.01,           # Initial learning rate
    patience=20,        # Early stopping patience
    save_period=10,     # Save checkpoint every N epochs

    # Data augmentation (good for small datasets)
    hsv_h=0.015,        # Hue augmentation
    hsv_s=0.7,          # Saturation augmentation
    hsv_v=0.4,          # Value augmentation
    degrees=10.0,       # Rotation degrees
    translate=0.1,      # Translation fraction
    scale=0.5,          # Scaling factor
    fliplr=0.5,         # Horizontal flip probability

    # Validation
    val=True,           # Validate during training
    plots=True,         # Generate training plots
)

print("Training completed!")


🚀 TRAINING YOLONANO MODEL
Using batch size: 16
Ultralytics 8.3.203 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=fish_dataset/data.yaml, degrees=10.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n_fish_v1, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True,

In [ ]:
# ============================================================================
# CELL 5B: Train Additional YOLO Models (Small, Medium, Large)
# ============================================================================

# Available YOLO model sizes
model_variants = {
    'small': 'yolov8s.pt',    # Good balance
    'medium': 'yolov8m.pt',   # Better accuracy
    'large': 'yolov8l.pt',    # High accuracy
}

# Train these models
models_to_train = ['small', 'medium', 'large']

for model_name in models_to_train:
    print(f"\n{'='*60}")
    print(f"🚀 TRAINING YOLO{model_name.upper()} MODEL")
    print('='*60)

    # Load the model
    model = YOLO(model_variants[model_name])

    # Batch sizes
    batch_sizes = {'small': 12, 'medium': 8, 'large': 4}
    batch_size = batch_sizes[model_name]
    print(f"Using batch size: {batch_size}")

    # Train the model
    results = model.train(
        data=f'{dataset_path}/data.yaml',
        epochs=50,
        imgsz=640,
        batch=batch_size,
        device=0,
        project='fish_detection_comparison',
        name=f'yolo{model_name}_fish',
        lr0=0.01,
        patience=15,
        save_period=10,
        val=True,
        plots=True,
    )

    print(f"✅ {model_name.upper()} training completed!")

print("\n🎉 ALL ADDITIONAL MODELS COMPLETED!")


🚀 TRAINING YOLOSMALL MODEL
Using batch size: 12
Ultralytics 8.3.203 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=12, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=fish_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolosmall_fish2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True,

In [ ]:
# ============================================================================
# CELL 6: Evaluate xLARGE Model Performance if IT WAS TRAINED
# ============================================================================

# Since trained_results is empty, find the trained model manually
model_path = "/content/fish_detection/yolov8n_fish_v1/weights/best.pt"

print("\n📊 MODEL EVALUATION RESULTS")
print("="*50)

# Load and validate the trained model
model = YOLO(model_path)
validation_results = model.val()

# Display results
print(f"Model: YOLOv8x (xlarge)")
print(f"mAP50: {validation_results.box.map50:.4f}")
print(f"mAP50-95: {validation_results.box.map:.4f}")
print(f"Precision: {validation_results.box.mp:.4f}")
print(f"Recall: {validation_results.box.mr:.4f}")
print(f"Model Size: {os.path.getsize(model_path) / 1024 / 1024:.1f} MB")

# Class-wise performance
if hasattr(validation_results.box, 'maps'):
    print("\nClass-wise mAP50:")
    for i, map_val in enumerate(validation_results.box.maps):
        print(f"  Class {i}: {map_val:.4f}")


📊 MODEL EVALUATION RESULTS
Ultralytics 8.3.201 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 112 layers, 68,129,346 parameters, 0 gradients, 257.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1473.5±847.1 MB/s, size: 59.1 KB)
val: Scanning /content/fish_dataset/valid/labels.cache... 88 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 88/88 178.6Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 6/6 0.9it/s 6.3s
                   all         88        368      0.774      0.769      0.776      0.448
             ancistrus         52         67      0.856      0.888      0.896      0.539
           black molly         48         64      0.895      0.859      0.901      0.576
       dalmatian molly          5          5      0.768        0.6      0.642      0.346
             gold fish         40         41       0.78       0.95      0.898      0.512
            gold

In [ ]:
# ============================================================================
# CELL 6: Compare All Four Models
# ============================================================================

import pandas as pd

print("\n📊 ALL MODELS COMPARISON")
print("="*80)

# Define all trained models
all_models = {
    'nano': '/content/fish_detection/yolov8n_fish_v1/weights/best.pt',
    'large': '/content/fish_detection_comparison/yololarge_fish/weights/best.pt',
    'medium': '/content/fish_detection_comparison/yolomedium_fish/weights/best.pt',
    'small': '/content/fish_detection_comparison/yolosmall_fish/weights/best.pt'
}

comparison_data = []

for model_name, model_path in all_models.items():
    if os.path.exists(model_path):
        model = YOLO(model_path)
        validation_results = model.val()

        comparison_data.append({
            'Model': f'YOLOv8{model_name[0]}',
            'mAP50': round(float(validation_results.box.map50), 4),
            'mAP50-95': round(float(validation_results.box.map), 4),
            'Precision': round(float(validation_results.box.mp), 4),
            'Recall': round(float(validation_results.box.mr), 4),
            'Size_MB': round(os.path.getsize(model_path) / 1024 / 1024, 1)
        })
    else:
        print(f"⚠️ Model not found: {model_path}")

# Display comparison
df = pd.DataFrame(comparison_data)
print(df.to_string(index=False))

# Find best model
if not df.empty:
    best_idx = df['mAP50'].idxmax()
    best_model = df.loc[best_idx, 'Model']
    print(f"\n🏆 BEST MODEL: {best_model} (mAP50: {df.loc[best_idx, 'mAP50']})")


📊 ALL MODELS COMPARISON
Ultralytics 8.3.203 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,007,208 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1744.1±348.6 MB/s, size: 60.7 KB)
val: Scanning /content/fish_dataset/valid/labels.cache... 116 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 116/116 220.4Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 8/8 2.4it/s 3.3s
                   all        116        584      0.726      0.736      0.755      0.465
             ancistrus         58         73      0.888      0.872      0.873      0.573
           black molly         63         94      0.849      0.819      0.886      0.575
       dalmatian molly         16         17      0.765      0.647      0.745       0.43
             gold fish         61         62      0.685      0.871      0.855      0.546
            gold mol

In [ ]:
# ============================================================================
# CELL 8: Create Deployment Package for All Models
# ============================================================================

import zipfile
import shutil
from datetime import datetime
import json

print("📦 Creating deployment package for all models...")
print("="*60)

# Define all trained models
all_models = {
  #  'xlarge': '/content/fish_detection/yolov8n_fish_v1/weights/best.pt',
  #  'large': '/content/fish_detection_comparison/yololarge_fish/weights/best.pt',
  #  'medium': '/content/fish_detection_comparison/yolomedium_fish/weights/best.pt',
  #  'small': '/content/fish_detection_comparison/yolosmall_fish/weights/best.pt'
  'nano': '/content/fish_detection/yolov8n_fish_v1/weights/best.pt',
}

# Create package directory
package_dir = "/content/all_fish_models_package"
os.makedirs(package_dir, exist_ok=True)

# Copy all models and export to ONNX
exported_models = {}
for model_name, model_path in all_models.items():
    if os.path.exists(model_path):
        print(f"Processing {model_name} model...")

        # Copy PyTorch model
        pt_filename = f'yolov8{model_name[0]}_fish.pt'
        shutil.copy2(model_path, os.path.join(package_dir, pt_filename))

        # Export to ONNX
        try:
            model = YOLO(model_path)
            onnx_path = model.export(format='onnx')
            onnx_filename = f'yolov8{model_name[0]}_fish.onnx'
            shutil.copy2(onnx_path, os.path.join(package_dir, onnx_filename))

            exported_models[model_name] = {
                'pytorch': pt_filename,
                'onnx': onnx_filename,
                'size_mb': round(os.path.getsize(model_path) / 1024 / 1024, 1)
            }
            print(f"✓ {model_name}: PyTorch + ONNX exported")

        except Exception as e:
            print(f"✗ {model_name} ONNX export failed: {e}")
            exported_models[model_name] = {'pytorch': pt_filename, 'onnx': None}
    else:
        print(f"⚠️ {model_name} model not found: {model_path}")

# Copy dataset config
shutil.copy2(f'{dataset_path}/data.yaml', os.path.join(package_dir, 'data.yaml'))

# Create deployment summary
summary = {
    'package_date': datetime.now().isoformat(),
    'models_included': exported_models,
    'dataset_path': dataset_path,
    'total_models': len([m for m in exported_models.values() if m.get('pytorch')])
}

with open(os.path.join(package_dir, 'models_summary.json'), 'w') as f:
    json.dump(summary, f, indent=2)

readme = """# Fish Detection Models Package

## Models Included
- YOLOv8s: Small model for balanced performance
- YOLOv8m: Medium model for better accuracy
- YOLOv8l: Large model for high accuracy
- YOLOv8x: Extra large model for best accuracy

## Pi 5 Deployment Recommendations
- Real-time tracking (>10 FPS): Use yolov8s_fish.onnx
- Balanced performance: Use yolov8m_fish.onnx
- Best accuracy: Use yolov8l_fish.onnx
- Periodic analysis: Use yolov8x_fish.onnx

## Usage
from ultralytics import YOLO
model = YOLO('yolov8s_fish.pt')
results = model('image.jpg')
"""

📦 Creating deployment package for all models...
Processing nano model...
Ultralytics 8.3.203 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/
Model summary (fused): 72 layers, 3,007,208 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from '/content/fish_detection/yolov8n_fish_v1/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 12, 8400) (6.0 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim>=0.1.67', 'onnxruntime-gpu'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 6.6s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.19.0 opset 22...
ONNX: slimming with onnxslim 0.1.69...
ONNX: export success ✅ 8.8s, saved as '/content/fish_detection/yolov8n_fish_v1/weights/best.onnx' (11.7 MB

In [ ]:
zip_path = "/content/nano_fish_model.zip"
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
  for root, dirs, files in os.walk(package_dir):
    for file in files:
      file_path = os.path.join(root, file)
      archive_name = os.path.relpath(file_path, package_dir)
      zipf.write(file_path, archive_name)
print(f"\n📦 Package created: {zip_path}")
print(f"📏 Package size: {os.path.getsize(zip_path) / 1024 / 1024:.1f} MB")
print(f"📊 Models included: {len(exported_models)}")


📦 Package created: /content/nano_fish_model.zip
📏 Package size: 15.5 MB
📊 Models included: 1


In [ ]:
from google.colab import files
# Download the package
try:
    files.download(zip_path)
    print("✅ Package downloaded successfully!")
except Exception as e:
    print(f"❌ Download failed: {e}")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Package downloaded successfully!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import shutil
drive_path = "/content/drive/MyDrive/fish_detection_model.zip"
shutil.copy2("/content/fish_detection_model.zip", drive_path)
print(f"✓ Model saved to Google Drive: fish_detection_model.zip")

Mounted at /content/drive
✓ Model saved to Google Drive: fish_detection_model.zip
